In [288]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import json
df = pd.read_csv('2020_now_merge_df.csv')

# tm 날짜
# ta 기온
# rn 강수량
# ws 풍속
# wd 풍향
# hm 습도
# pa 현지기압
# ss 일조
# dc10Tca 전운량
# dmstMtphNo 현상번호 - 눈 비인지 알수있음
# aplYmd 날짜
# lareaNm 서울특별시 고정
# mareaNm 중구 고정
# wetrCd 날씨코드
# wetrTxt 날씨코드설명

In [289]:
df.head(1)

,tm,ta,rn,ws,wd,hm,pa,ss,dc10Tca,dmstMtphNo,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,2019-12-29 00:00,1.4,NaN,1.8,50.0,57,1018.2,NaN,3.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비


In [290]:
# 강수랑 nan값은 다 0으로 저장
df["rn"] = df["rn"].fillna(0)

In [291]:
df.isna().sum() 

tm                0
ta                0
rn                0
ws               70
wd               70
hm                0
pa               10
ss            17165
dc10Tca          20
dmstMtphNo    27246
aplYmd            0
lareaNm           0
mareaNm           0
wetrCd            0
wetrTxt           0
dtype: int64

In [292]:
# 일조를 어디다가 쓰나요? => 일단지움
# 혹시모르니
ss_no_df = df.drop(columns=["ss"],axis=1).copy()

In [293]:
ss_no_df.isna().sum()

tm                0
ta                0
rn                0
ws               70
wd               70
hm                0
pa               10
dc10Tca          20
dmstMtphNo    27246
aplYmd            0
lareaNm           0
mareaNm           0
wetrCd            0
wetrTxt           0
dtype: int64

In [294]:
ss_no_df[ss_no_df["ws"].isna()]

,tm,ta,rn,ws,wd,hm,pa,dc10Tca,dmstMtphNo,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
9011,2021-01-07 11:00,-14.8,0.0,NaN,NaN,48,1016.9,0.0,NaN,20210107,서울특별시,종로구,1.0,맑음
9012,2021-01-07 12:00,-13.8,0.0,NaN,NaN,45,1016.4,0.0,NaN,20210107,서울특별시,종로구,1.0,맑음
33793,2023-12-10 01:00,11.6,0.0,NaN,NaN,87,1010.4,10.0,19.0,20231210,서울특별시,종로구,9.0,구름많음/비/안개
33794,2023-12-10 02:00,11.4,0.0,NaN,NaN,86,1010.7,9.0,19.0,20231210,서울특별시,종로구,9.0,구름많음/비/안개
33795,2023-12-10 03:00,10.8,0.0,NaN,NaN,82,1010.9,2.0,19.0,20231210,서울특별시,종로구,9.0,구름많음/비/안개
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36037,2024-03-12 13:00,7.8,0.0,NaN,NaN,76,1004.1,9.0,NaN,20240312,서울특별시,종로구,9.0,구름많음/비/안개/우박
36038,2024-03-12 14:00,7.9,0.0,NaN,NaN,73,1003.6,10.0,1.0,20240312,서울특별시,종로구,9.0,구름많음/비/안개/우박
36039,2024-03-12 15:00,9.4,0.0,NaN,NaN,67,1003.3,8.0,1.0,20240312,서울특별시,종로구,9.0,구름많음/비/안개/우박
36040,2024-03-12 16:00,9.2,0.0,NaN,NaN,68,1003.6,7.0,1301.0,20240312,서울특별시,종로구,9.0,구름많음/비/안개/우박


In [295]:
# 풍향을 예측하는거 index 저장
wd_index = ss_no_df[ss_no_df["wd"].isna()].index

In [296]:
wd_index

Index([ 9011,  9012, 33793, 33794, 33795, 33796, 33797, 33798, 33799, 33800,
       33801, 33802, 33803, 33804, 33805, 33806, 33807, 33808, 33809, 33810,
       33811, 33812, 33813, 33814, 33815, 33816, 33817, 33818, 33819, 33820,
       33821, 33822, 33823, 33824, 33825, 33826, 33827, 33828, 33829, 36004,
       36005, 36006, 36007, 36008, 36009, 36010, 36018, 36019, 36020, 36021,
       36022, 36023, 36024, 36025, 36026, 36027, 36028, 36029, 36030, 36031,
       36032, 36033, 36034, 36035, 36036, 36037, 36038, 36039, 36040, 36041],
      dtype='int64')

In [297]:
for i in range(len(wd_index) - 1):
    gap = wd_index[i + 1] - wd_index[i]
    print(f"인덱스 {wd_index[i]}와 인덱스 {wd_index[i + 1]} 사이의 간격: {gap}")
# 9011 9012 앞뒤가 비어있지않은값만 보기

인덱스 9011와 인덱스 9012 사이의 간격: 1
인덱스 9012와 인덱스 33793 사이의 간격: 24781
인덱스 33793와 인덱스 33794 사이의 간격: 1
인덱스 33794와 인덱스 33795 사이의 간격: 1
인덱스 33795와 인덱스 33796 사이의 간격: 1
인덱스 33796와 인덱스 33797 사이의 간격: 1
인덱스 33797와 인덱스 33798 사이의 간격: 1
인덱스 33798와 인덱스 33799 사이의 간격: 1
인덱스 33799와 인덱스 33800 사이의 간격: 1
인덱스 33800와 인덱스 33801 사이의 간격: 1
인덱스 33801와 인덱스 33802 사이의 간격: 1
인덱스 33802와 인덱스 33803 사이의 간격: 1
인덱스 33803와 인덱스 33804 사이의 간격: 1
인덱스 33804와 인덱스 33805 사이의 간격: 1
인덱스 33805와 인덱스 33806 사이의 간격: 1
인덱스 33806와 인덱스 33807 사이의 간격: 1
인덱스 33807와 인덱스 33808 사이의 간격: 1
인덱스 33808와 인덱스 33809 사이의 간격: 1
인덱스 33809와 인덱스 33810 사이의 간격: 1
인덱스 33810와 인덱스 33811 사이의 간격: 1
인덱스 33811와 인덱스 33812 사이의 간격: 1
인덱스 33812와 인덱스 33813 사이의 간격: 1
인덱스 33813와 인덱스 33814 사이의 간격: 1
인덱스 33814와 인덱스 33815 사이의 간격: 1
인덱스 33815와 인덱스 33816 사이의 간격: 1
인덱스 33816와 인덱스 33817 사이의 간격: 1
인덱스 33817와 인덱스 33818 사이의 간격: 1
인덱스 33818와 인덱스 33819 사이의 간격: 1
인덱스 33819와 인덱스 33820 사이의 간격: 1
인덱스 33820와 인덱스 33821 사이의 간격: 1
인덱스 33821와 인덱스 33822 사이의 간격: 1
인덱스 33822와 인덱스 33823 사이의 간격: 1
인덱스 338

In [298]:
import numpy as np
from sklearn.linear_model import LinearRegression
# 풍향 예측하기
# 9011 9012 앞뒤가 비어있지않은값만 예측

In [299]:
predict_array = ss_no_df.loc[9006:9017,"wd"]

In [300]:
predict_array_df = pd.DataFrame(predict_array)


In [301]:
predict_array_df.to_csv('predict_array_df.csv', index=False, encoding='utf-8-sig')

In [302]:
import joblib
data = np.array(predict_array).reshape(-1, 1)
model = LinearRegression()
x_train = np.where(~np.isnan(data))[0].reshape(-1, 1)
y_train = data[~np.isnan(data)].reshape(-1, 1)
model.fit(x_train, y_train)
X_pred = np.where(np.isnan(data))[0].reshape(-1, 1)
y_pred = model.predict(X_pred)
joblib.dump(model, 'linear_regression_model.joblib')
print("예측 결과:", y_pred.flatten())

예측 결과: [279.54385965 280.45614035]


In [303]:
ss_no_df.loc[9011,"wd"] = 279.54385965

In [304]:
ss_no_df.loc[9012,"wd"] = 280.45614035

In [305]:
# 풍향을 예측하는거 index 저장
ws_index = ss_no_df[ss_no_df["ws"].isna()].index

In [306]:
ws_index

Index([ 9011,  9012, 33793, 33794, 33795, 33796, 33797, 33798, 33799, 33800,
       33801, 33802, 33803, 33804, 33805, 33806, 33807, 33808, 33809, 33810,
       33811, 33812, 33813, 33814, 33815, 33816, 33817, 33818, 33819, 33820,
       33821, 33822, 33823, 33824, 33825, 33826, 33827, 33828, 33829, 36004,
       36005, 36006, 36007, 36008, 36009, 36010, 36018, 36019, 36020, 36021,
       36022, 36023, 36024, 36025, 36026, 36027, 36028, 36029, 36030, 36031,
       36032, 36033, 36034, 36035, 36036, 36037, 36038, 36039, 36040, 36041],
      dtype='int64')

In [307]:
for i in range(len(ws_index) - 1):
    gap = ws_index[i + 1] - ws_index[i]
    # print(f"인덱스 {ws_index[i]}와 인덱스 {ws_index[i + 1]} 사이의 간격: {gap}")
# 9011 9012 앞뒤가 비어있지않은값만 보기

In [308]:
predict_array = ss_no_df.loc[9006:9017,"ws"]
data = np.array(predict_array).reshape(-1, 1)
model = LinearRegression()
x_train = np.where(~np.isnan(data))[0].reshape(-1, 1)
y_train = data[~np.isnan(data)].reshape(-1, 1)
model.fit(x_train, y_train)
X_pred = np.where(np.isnan(data))[0].reshape(-1, 1)
y_pred = model.predict(X_pred)
print("예측 결과:", y_pred.flatten())

ss_no_df.loc[9011,"ws"] = 4.59649123
ss_no_df.loc[9012,"ws"] = 4.56350877

예측 결과: [4.59649123 4.56350877]


In [309]:
ss_no_df.isna().sum()

tm                0
ta                0
rn                0
ws               68
wd               68
hm                0
pa               10
dc10Tca          20
dmstMtphNo    27246
aplYmd            0
lareaNm           0
mareaNm           0
wetrCd            0
wetrTxt           0
dtype: int64

In [310]:
# 기압을 예측하는거 index 저장
pa_index = ss_no_df[ss_no_df["pa"].isna()].index

In [311]:
for i in range(len(pa_index) - 1):
    gap = pa_index[i + 1] - pa_index[i]
    print(f"인덱스 {pa_index[i]}와 인덱스 {pa_index[i + 1]} 사이의 간격: {gap}")
# 8033  8034  앞뒤가 비어있지않은값만 보기

인덱스 6210와 인덱스 6211 사이의 간격: 1
인덱스 6211와 인덱스 6212 사이의 간격: 1
인덱스 6212와 인덱스 6213 사이의 간격: 1
인덱스 6213와 인덱스 6214 사이의 간격: 1
인덱스 6214와 인덱스 6215 사이의 간격: 1
인덱스 6215와 인덱스 6216 사이의 간격: 1
인덱스 6216와 인덱스 6217 사이의 간격: 1
인덱스 6217와 인덱스 8033 사이의 간격: 1816
인덱스 8033와 인덱스 8034 사이의 간격: 1


In [312]:
predict_array = ss_no_df.loc[8027:8040,"pa"]
data = np.array(predict_array).reshape(-1, 1)
model = LinearRegression()
x_train = np.where(~np.isnan(data))[0].reshape(-1, 1)
y_train = data[~np.isnan(data)].reshape(-1, 1)
model.fit(x_train, y_train)
X_pred = np.where(np.isnan(data))[0].reshape(-1, 1)
y_pred = model.predict(X_pred)
print("예측 결과:", y_pred.flatten())

예측 결과: [1015.23061674 1015.46938326]


In [313]:
ss_no_df.loc[8033,"pa"] = 1015.23061674 
ss_no_df.loc[8034,"pa"] = 1015.46938326

In [314]:
ss_no_df.isna().sum()

tm                0
ta                0
rn                0
ws               68
wd               68
hm                0
pa                8
dc10Tca          20
dmstMtphNo    27246
aplYmd            0
lareaNm           0
mareaNm           0
wetrCd            0
wetrTxt           0
dtype: int64

In [315]:
# 전운량을 예측하는거 index 저장
dc10Tca_index = ss_no_df[ss_no_df["dc10Tca"].isna()].index

In [316]:
dc10Tca_index

Index([  139,  1249,  1250,  1393,  4342,  4343,  4582,  4583,  4699,  4700,
        4702,  4703,  4705,  4706,  4727,  4729,  5516,  7892, 14179, 14227],
      dtype='int64')

In [317]:
for i in range(len(dc10Tca_index) - 1):
    gap = dc10Tca_index[i + 1] - dc10Tca_index[i]
    # print(f"인덱스 {dc10Tca_index[i]}와 인덱스 {dc10Tca_index[i + 1]} 사이의 간격: {gap}")
dc10Tca_predict_list = [139, 1249, 1250,1393,4342, 4343, 4582, 4583, 4699,4727,4729,5516,7892,14179,14227]
# 139 1249 1250 1393 4342 4343 4582 4583 4699  앞뒤가 비어있지않은값만 보기

In [318]:
for i in range(len(dc10Tca_predict_list)):
    predict_array = ss_no_df.loc[dc10Tca_predict_list[i]-6:dc10Tca_predict_list[i]+6,"dc10Tca"]
    data = np.array(predict_array).reshape(-1, 1)
    model = LinearRegression()
    x_train = np.where(~np.isnan(data))[0].reshape(-1, 1)
    y_train = data[~np.isnan(data)].reshape(-1, 1)
    model.fit(x_train, y_train)
    X_pred = np.where(np.isnan(data))[0].reshape(-1, 1)
    y_pred = model.predict(X_pred)
    # print("예측 결과:", y_pred.flatten())
    # print(len(y_pred))
    ss_no_df.loc[dc10Tca_predict_list[i],"dc10Tca"] = y_pred[0]

In [319]:
ss_no_df['wetrTxt'] = ss_no_df['wetrTxt'].astype(str)

In [320]:
type(ss_no_df.loc[0,"wetrTxt"])

str

In [321]:
# dmstMtphNo 일단 naver_data로 채워넣기
ss_no_df["wetrSplit"] = ss_no_df["wetrTxt"].str.split("/")

In [322]:
ss_no_df.shape

(37776, 15)

In [323]:
all_values_set = set().union(*ss_no_df['wetrSplit'])

In [324]:
all_values_set

{'구름많음',
 '구름조금',
 '눈',
 '맑음',
 '비',
 '소나기',
 '소낙눈',
 '안개',
 '우박',
 '이슬비',
 '진눈개비',
 '천둥번개',
 '황사',
 '흐림'}

In [325]:
weather_dict = {"구름많음" : "91" ,
               "구름조금" : "91",
               "눈" : "05",
               "맑음" : "90",
               "비" : "01",
               "소나기" : "01",
               "소낙눈" : "05",
               "이슬비" : "01",
               "진눈개비" : "05",
               "흐림" : "92",
               }

In [326]:
ss_no_df = ss_no_df.reset_index(drop=True)

In [327]:
all_weather_list=[]

In [328]:
for i in range(len(ss_no_df)):
    new_list = []
    weather_list = ss_no_df.loc[i, "wetrSplit"]
    for weather in weather_list:
        for key, value in weather_dict.items():
            if key == weather:
                new_list.append(value)
    all_weather_list.append(new_list)
    # ss_no_df.loc[i, "wetrList"] = new_list
    # ss_no_df.loc[i, "wetrList"] = new_list


In [329]:
ss_no_df["new_list"] = None

In [330]:
ss_no_df['new_list'] = all_weather_list

In [331]:
ss_no_df

,tm,ta,rn,ws,wd,hm,pa,dc10Tca,dmstMtphNo,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt,wetrSplit,new_list
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"[흐림, 비]","[92, 01]"
1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"[흐림, 비]","[92, 01]"
2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"[흐림, 비]","[92, 01]"
3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"[흐림, 비]","[92, 01]"
4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,NaN,20191229,서울특별시,종로구,9.0,흐림/비,"[흐림, 비]","[92, 01]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37771,2024-05-23 19:00,22.5,0.0,3.5,270.0,55,999.5,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개,"[맑음, 안개]",[90]
37772,2024-05-23 20:00,20.5,0.0,3.0,290.0,68,1000.3,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개,"[맑음, 안개]",[90]
37773,2024-05-23 21:00,18.6,0.0,2.8,270.0,73,1001.0,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개,"[맑음, 안개]",[90]
37774,2024-05-23 22:00,17.7,0.0,2.9,290.0,79,1001.6,0.0,NaN,20240523,서울특별시,종로구,17.0,맑음/안개,"[맑음, 안개]",[90]


In [332]:
ss_no_df.to_csv('data_processing1.csv', index=False, encoding='utf-8-sig')